In [1]:
import re
from bs4 import BeautifulSoup
import pandas as pd
import requests as rq

url = f"https://comp.fnguide.com/SVO2/ASP/SVD_finance.asp?pGB=1&gicode=A005930"
data = pd.read_html(url, displayed_only=False)

In [2]:
def get_fiscal_date(url):
    page_data = rq.get(url)
    page_data_html = BeautifulSoup(page_data.content, "html.parser")
    fiscal_data = page_data_html.select("div.corp_group1 > h2")
    fiscal_data_text = fiscal_data[1].text
    fiscal_data_text = re.findall("[0-9]+", fiscal_data_text)
    return fiscal_data_text


def clean_fs(df, ticker, frequency):
    df = df[~df.loc[:, ~df.columns.isin(["계정"])].isna().all(axis=1)]
    df.loc[:, "계정"] = df["계정"].replace(
        {"계산에 참여한 계정 펼치기": ""}, regex=True
    )
    df = df.drop_duplicates(["계정"], keep="first")
    df = pd.melt(df, id_vars="계정", var_name="기준일", value_name="값")
    # df["계정"] = df["계정"].replace({"계산에 참여한 계정 펼치기": ""}, regex=True)
    df = df[~pd.isnull(df["값"])]
    df["기준일"] = (
        pd.to_datetime(df["기준일"], format="%Y/%m") + pd.tseries.offsets.MonthEnd()
    )
    df["기준일"] = df["기준일"].dt.strftime("%Y%m%d")
    type
    df["종목코드"] = ticker
    df["공시구분"] = frequency

    return df


fiscal_data_text = get_fiscal_date(url)

In [18]:
data_fs_y = pd.concat(
    [
        data[0].iloc[:, ~data[0].columns.str.contains("전년동기")],
        data[2],
        data[4],
    ]
)

In [19]:
data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})
# 결산월 뒤 분기데이터 삭제
data_fs_y = data_fs_y.loc[
    :,
    (data_fs_y.columns == "계정") | (data_fs_y.columns.str[-2:].isin(fiscal_data_text)),
]

In [10]:
df = data_fs_y

# replace 후에 중복제거
df = df[~df.loc[:, ~df.columns.isin(["계정"])].isna().all(axis=1)]  # null값 제거
df.loc[:, "계정"] = df["계정"].replace({"계산에 참여한 계정 펼치기": ""}, regex=True)
df = df.drop_duplicates(["계정"], keep="first")
df = pd.melt(df, id_vars="계정", var_name="기준일", value_name="값")
df = df[~pd.isnull(df["값"])]

In [20]:
data_fs_q = pd.concat(
    [
        data[1].iloc[:, ~data[0].columns.str.contains("전년동기")],
        data[3],
        data[5],
    ]
)

In [40]:
data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: "계정"})

In [41]:
df = data_fs_q

# replace 후에 중복제거
df = df[~df.loc[:, ~df.columns.isin(["계정"])].isna().all(axis=1)]  # null값 제거
df.loc[:, "계정"] = df["계정"].replace({"계산에 참여한 계정 펼치기": ""}, regex=True)
df = df.drop_duplicates(["계정"], keep="first")
df = pd.melt(df, id_vars="계정", var_name="기준일", value_name="값")
df = df[~pd.isnull(df["값"])]